In [2]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
from openpyxl import load_workbook

# Define the parameters
short_city=['Si','Ro','Mi']
long_city=['Siracusa','Roma','Milano']

for c in range(3):
    for user in ['A','B','C','D']:
        city=short_city[c]
        city_extended=long_city[c]
        user_name=f'User{user}'
        scenario_name=f'Base_{city}_{user}'
        # Importing from esm database.db
        def import_from_db(user_name,db_name):
            conn = sqlite3.connect(db_name)

            query_cost_op_tot = "SELECT * FROM cost_op_tot WHERE s_names=?"
            cost_op_tot = pd.read_sql_query(query_cost_op_tot, conn, params=(user_name,))

            query_cost_period = "SELECT * FROM cost_period"
            cost_period = pd.read_sql_query(query_cost_period, conn)

            query_active_techs = "SELECT * FROM active_techs WHERE s_names=?"
            active_techs = pd.read_sql_query(query_active_techs, conn, params=(user_name,))

            query_cost_cap = "SELECT * FROM cost_cap"
            cost_cap = pd.read_sql_query(query_cost_cap, conn)

            query_cap_tot = "SELECT * FROM cap_tot WHERE s_names=?"
            cap_tot = pd.read_sql_query(query_cap_tot, conn, params=(user_name,))

            query_cost_inv = "SELECT * FROM cost_inv WHERE s_names=?"
            cost_inv = pd.read_sql_query(query_cost_inv, conn, params=(user_name,))

            query_new_units = "SELECT * FROM new_units WHERE s_names=?"
            new_units = pd.read_sql_query(query_new_units, conn, params=(user_name,))

            query_cap_new = "SELECT * FROM cap_new WHERE s_names=?"
            cap_new = pd.read_sql_query(query_cap_new, conn, params=(user_name,))

            # For scalar values
            query_disc_frac = "SELECT * FROM disc_frac WHERE s_names=?"
            disc_frac_pd = pd.read_sql_query(query_disc_frac, conn, params=(user_name,))
            disc_frac = disc_frac_pd['values'].iloc[0]

            query_insulation = "SELECT * FROM insulation WHERE s_names=?"
            insulation_pd = pd.read_sql_query(query_insulation, conn, params=(user_name,))
            insulation = insulation_pd['values'].iloc[0]

            query_TI_cost = "SELECT * FROM TI_cost WHERE s_names=?"
            TI_cost_pd = pd.read_sql_query(query_TI_cost, conn, params=(user_name,))
            TI_cost = TI_cost_pd['values'].iloc[0]

            query_CO2 = "SELECT * FROM CO2_tot WHERE s_names=?"
            CO2_tot_pd = pd.read_sql_query(query_CO2, conn, params=(user_name,))
            CO2_tot=CO2_tot_pd['values'].iloc[0]

            query_Y = "SELECT * FROM Y WHERE s_names=?"  
            Y = pd.read_sql_query(query_Y, conn, params=(user_name,))
            query_Q = "SELECT * FROM Q WHERE s_names=?"
            Q = pd.read_sql_query(query_Q, conn, params=(user_name,))
            query_X = "SELECT * FROM X WHERE s_names=?"
            X = pd.read_sql_query(query_X, conn, params=(user_name,))
            query_Xt = "SELECT * FROM X_t WHERE s_names=?"
            Xt = pd.read_sql_query(query_Xt, conn, params=(user_name,))
            query_DPT = "SELECT * FROM days_per_type"
            DPT = pd.read_sql_query(query_DPT, conn)
            query_SOC = "SELECT * FROM SOC WHERE s_names=?"
            SOC = pd.read_sql_query(query_SOC, conn, params=(user_name,))

            conn.close()

            return cost_op_tot, cost_period, active_techs, cost_cap, cap_tot, cost_inv, new_units, cap_new, disc_frac, insulation, TI_cost, CO2_tot, Y, Q, X, Xt, DPT, SOC

        cost_op_tot, cost_period, active_techs, cost_cap, cap_tot, cost_inv, new_units, cap_new, disc_frac, insulation, TI_cost, CO2_tot, Y, Q, X, Xt, DPT, SOC =import_from_db(user_name,f'{city_extended}/database.db')

        #Definitions for time
        seasons = ['wi', 'mc', 'mw', 'su']
        season_names = {'wi': 'Winter', 'mc': 'Mid-cold', 'mw': 'Mid-warm', 'su': 'Summer'}

        #Isolate peak
        peak_Y= Y[Y['th_names'] == 'peak']
        Y=Y[Y['th_names'] != 'peak']
        peak_Q= Q[Q['th_names'] == 'peak']
        Q=Q[Q['th_names'] != 'peak']
        peak_X=X[X['th_names'] == 'peak']
        X=X[X['th_names'] != 'peak']
        peak_Xt=Xt[Xt['th_names'] == 'peak']
        Xt=Xt[Xt['th_names'] != 'peak']
        SOC=SOC[SOC['th_names'] != 'peak']
        DPT=DPT[DPT['days_names'] != 'peak']

        for db in [Y,Q,X,Xt,SOC]:
            db.index=range(len(db))
            db.loc[:,'season']=db['th_names'].str[4:6]
            db['hour'] = db['th_names'].str[1:3].astype(int)
            db['day'] = db['th_names'].str[-2:].astype(int)
            db['time'] = (db['day'] - 1) * 24 + db['hour']

        #Defining values to export in results.xlsx
        activity_costs=sum(cost_op_tot['values'])
        periodic_costs=sum((cost_period['values']+(cost_cap['values']*cap_tot['values']))*active_techs['values'])
        bill_one_year=activity_costs+periodic_costs
        operation_costs_discounted=bill_one_year*(sum(disc_frac ** x for x in range(1, 11)))
        investment_costs=sum(cost_inv['values']*new_units['values'])
        NPC=operation_costs_discounted+investment_costs

        cost_EE_period=cost_period.loc[cost_period['t_names']=='National Grid EE']['values'].values[0]+cost_cap.loc[cost_cap['t_names']=='National Grid EE']['values'].values[0]*cap_tot.loc[cap_tot['t_names']=='National Grid EE']['values'].values[0]
        cost_EE=cost_op_tot.loc[cost_op_tot['a_names']=='National Grid import']['values'].values[0]+cost_op_tot.loc[cost_op_tot['a_names']=='National Grid export']['values'].values[0]+cost_EE_period

        cost_NG_period=active_techs[active_techs['t_names']=='Gas National Grid']['values'].values[0]*cost_period.loc[cost_period['t_names']=='Gas National Grid']['values'].values[0]+cost_cap.loc[cost_cap['t_names']=='Gas National Grid']['values'].values[0]*cap_tot.loc[cap_tot['t_names']=='Gas National Grid']['values'].values[0]
        cost_NG=cost_op_tot.loc[cost_op_tot['a_names']=='Natural Gas supply grid']['values'].values[0]+cost_NG_period

        def compute_daily(df):
            #Group by day and season
            df = df.groupby(['day', 'season']).agg(consumption=('values', 'sum')).reset_index()
            #Fix the order
            season_order = ['wi', 'mc', 'mw', 'su']
            df['season'] = pd.Categorical(df['season'], categories=season_order, ordered=True)
            df_sorted = df.sort_values(by=['season', 'day'])
            df_sorted.index = range(len(df_sorted))
            return df_sorted

        Q_EE=Q[Q['n_names']=='EE']
        Q_EE_daily = compute_daily(Q_EE)
        Q_EE_daily['consumption'] = Q_EE_daily['consumption']*DPT['values']
        Q_EE_tot=sum(Q_EE_daily['consumption'])

        X_EE_daily = compute_daily(X[X['a_names']=='National Grid import'])
        X_EE_daily['consumption'] = X_EE_daily['consumption']*DPT['values']

        PV_prod=X[X['a_names']=='PV']
        X_PV_daily = compute_daily(PV_prod)
        X_PV_daily['consumption'] = X_PV_daily['consumption']*DPT['values']

        PV_sell=X[X['a_names']=='National Grid export']
        X_sell_daily = compute_daily(PV_sell)
        X_sell_daily['consumption'] = X_sell_daily['consumption']*DPT['values']

        Q_NG_daily = compute_daily(Q[Q['n_names']=='Natural Gas'])
        Q_NG_tot=sum(Q_NG_daily['consumption']*DPT['values'])

        #Exporting results for this case
        results=pd.DataFrame(columns=['NPC','opex','capex','inst_PV','inst_GridEE','inst_StorageEE','inst_GasBoiler','inst_HP','inst_AC','inst_StorageHW','inst_rad','inst_floor','inst_induction','inst_GasStove','Q_EE','Q_NG','EE_in','EE_out','EE_PV','cash_EEin','cash_EEout','cash_EEtot','cash_NGin','cash_NGtot','CO2'])

        results.at[scenario_name,'NPC']=round(NPC,2)
        results.at[scenario_name,'opex']=round(bill_one_year,2)
        results.at[scenario_name,'capex']=round(investment_costs,2)
        results.at[scenario_name,'inst_PV']=round(cap_new.loc[cap_new['t_names']=='PV sys']['values'].values[0],2)
        results.at[scenario_name,'inst_GridEE']=round(cap_new.loc[cap_new['t_names']=='National Grid EE']['values'].values[0],2)
        results.at[scenario_name,'inst_StorageEE']=round(cap_new.loc[cap_new['t_names']=='Storage EE']['values'].values[0],2)
        results.at[scenario_name,'inst_GasBoiler']=round(cap_new.loc[cap_new['t_names']=='Gas Boiler']['values'].values[0],2)
        results.at[scenario_name,'inst_HP']=round(cap_new.loc[cap_new['t_names']=='HP']['values'].values[0],2)
        results.at[scenario_name,'inst_AC']=round(cap_new.loc[cap_new['t_names']=='AC']['values'].values[0],2)
        results.at[scenario_name,'inst_StorageHW']=round(cap_new.loc[cap_new['t_names']=='Storage HW']['values'].values[0],2)
        results.at[scenario_name,'inst_rad']=round(cap_new.loc[cap_new['t_names']=='Radiators']['values'].values[0],2)
        results.at[scenario_name,'inst_floor']=round(cap_new.loc[cap_new['t_names']=='Floor heating']['values'].values[0],2)
        results.at[scenario_name,'inst_induction']=round(cap_new.loc[cap_new['t_names']=='Induction stove']['values'].values[0],2)
        results.at[scenario_name,'inst_GasStove']=round(cap_new.loc[cap_new['t_names']=='Gas stove']['values'].values[0],2)
        results.at[scenario_name,'Q_EE']=round(Q_EE_tot,2)
        results.at[scenario_name,'Q_NG']=round(Q_NG_tot,2)
        results.at[scenario_name,'EE_in']=round(sum(X_EE_daily['consumption']),2)
        results.at[scenario_name,'EE_out']=round(sum(X_sell_daily['consumption']),2)
        results.at[scenario_name,'EE_PV']=round(sum(X_PV_daily['consumption']),2)
        results.at[scenario_name,'cash_EEin']=round(cost_op_tot.loc[cost_op_tot['a_names']=='National Grid import']['values'].values[0],2)
        results.at[scenario_name,'cash_EEout']=round(cost_op_tot.loc[cost_op_tot['a_names']=='National Grid export']['values'].values[0],2)
        results.at[scenario_name,'cash_EEtot']=round(cost_EE,2)
        results.at[scenario_name,'cash_NGin']=round(cost_op_tot.loc[cost_op_tot['a_names']=='Natural Gas supply grid']['values'].values[0],2)
        results.at[scenario_name,'cash_NGtot']=round(cost_NG,2)
        results.at[scenario_name,'CO2']=round(CO2_tot,4)


        file_path = "/Users/cami/Documents/GitHub/pyesm_thesis/tesi_camilla/casi_MarioU/results_database.xlsx"  # Sostituisci con il percorso corretto del file
        book = load_workbook(file_path)
        # Trova l'ultima riga occupata nel foglio
        sheet_name = "Sheet1"
        sheet = book["Sheet1"]
        last_row = sheet.max_row

        # Scrivi i dati usando l'ExcelWriter in modalità append
        with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            results.to_excel(writer, sheet_name=sheet_name, index=True, header=False, startrow=last_row)